In [ ]:
from pipelines.sdxl_pipeline import StableDiffusionXLRegionalAttnPipeline
from regional_attention.diffusers_unet import UNet2DConditionModel
from inference_on_dataset import NEGATIVE_PROMPTS
import json
import numpy as np
import torch
unet = UNet2DConditionModel.from_pretrained(
    'cplusx/rich_context_l2i_sdxl', subfolder='unet'
) # since we use a custom UNet model, if caling pipe.from_pretrained directly, the diffusers will raise error saying that the unet model is not found, so we create it first and pass it to the pipeline
pipe = StableDiffusionXLRegionalAttnPipeline.from_pretrained(
    "cplusx/rich_context_l2i_sdxl", unet=unet,
).to('cuda')

In [8]:
height = width = 512
GUIDANCE_SCALE = 5.0
caption = '' # You can leave it blank or give it a global caption if you have
object_file = 'demos/100002558.json'
with open(object_file, 'r') as f:
    objects = json.load(f)
    bboxes = np.array(objects['bboxes'])
    labels = objects['labels']

In [ ]:
with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.float16):
    generated = pipe(
        prompt=caption,
        negative_prompt=NEGATIVE_PROMPTS,
        height=height, width=width,
        boxes=bboxes,
        labels=labels,
        scheduled_sampling_beta=1,
        num_inference_steps=50,
        guidance_scale=GUIDANCE_SCALE
    )

In [ ]:
generated[0][0]